In [1]:
import pandas as pd
pd.options.display.max_columns = 110

import numpy as np
import re
import string
import sys
import csv
import nltk
from nltk.corpus import stopwords

from bs4 import BeautifulSoup
import spacy
import unidecode
from word2number import w2n


In [2]:
#import data

df_origine = pd.read_csv('./data/User_Netflix_squidgame_02.csv')

In [3]:
df_origine.head()

,User,Tweet ID,Time,Text
0,ahmedjl2x,1.450000e+18,2021-10-12 06:57:09+00:00,Late from the trend but still wanna try...my e...
1,ahmedjl2x,1.450000e+18,2021-10-12 06:57:05+00:00,The whole game could have been avoided ☠️ #Squ...
2,ahmedjl2x,1.450000e+18,2021-10-12 06:56:17+00:00,I've just watched episode S01 E08 of Squid Ga...
3,ahmedjl2x,1.450000e+18,2021-10-12 06:55:35+00:00,Late from the trend but still wanna try...my e...
4,ahmedjl2x,1.450000e+18,2021-10-12 06:54:32+00:00,There are already more than 10 games on SQUID ...


In [4]:
df=df_origine.drop(columns=['User', 'Tweet ID', 'Time'],axis=1)
df.head()

,Text
0,Late from the trend but still wanna try...my e...
1,The whole game could have been avoided ☠️ #Squ...
2,I've just watched episode S01 E08 of Squid Ga...
3,Late from the trend but still wanna try...my e...
4,There are already more than 10 games on SQUID ...


In [22]:
df.shape

(167, 2)

In [5]:
# To remove '@names' from the tweets
def remove_pattern(text, pattern_regex):
    r = re.findall(pattern_regex, text)
    for i in r:
        text = re.sub(i, '', text)
    return text 

# To keep cleaned tweets in a new column called 'tidy_tweets'
df['Text'] = np.vectorize(remove_pattern)(df['Text'], "@[\w]* | RT")
df.head(10)

,Text
0,Late from the trend but still wanna try...my e...
1,The whole game could have been avoided ☠️ #Squ...
2,I've just watched episode S01 E08 of Squid Ga...
3,Late from the trend but still wanna try...my e...
4,There are already more than 10 games on SQUID ...
5,Late from the trend but still wanna try...my e...
6,"I Hated This Show At First. Horrible Acting, B..."
7,Amazing Job again! love the !! Trippy bunny N...
8,Whoever created squid games \nGO\nTO\nHELL .\n...
9,Una nueva y contundente patada a la proverbial...


In [6]:
# To remove the links(http | https)
cleaned_tweets = []

for index, row in df.iterrows():
    # Here we are filtering out all the words that contains link
    words_without_links = [word for word in row.Text.split() if 'http' not in word]
    cleaned_tweets.append(' '.join(words_without_links))

df['Text'] = cleaned_tweets
df.head(10)

,Text
0,Late from the trend but still wanna try...my e...
1,The whole game could have been avoided ☠️ #Squ...
2,I've just watched episode S01 E08 of Squid Gam...
3,Late from the trend but still wanna try...my e...
4,There are already more than 10 games on SQUID ...
5,Late from the trend but still wanna try...my e...
6,"I Hated This Show At First. Horrible Acting, B..."
7,Amazing Job again! love the !! Trippy bunny NF...
8,Whoever created squid games GO TO HELL . #Squi...
9,Una nueva y contundente patada a la proverbial...


In [7]:
# To remove tweets with empty text
df = df[df['Text']!='']
df.head()

,Text
0,Late from the trend but still wanna try...my e...
1,The whole game could have been avoided ☠️ #Squ...
2,I've just watched episode S01 E08 of Squid Gam...
3,Late from the trend but still wanna try...my e...
4,There are already more than 10 games on SQUID ...


In [8]:
# To drop duplicate rows
df = df.drop_duplicates(subset=['Text'], keep=False)
df.head()

,Text
0,Late from the trend but still wanna try...my e...
1,The whole game could have been avoided ☠️ #Squ...
2,I've just watched episode S01 E08 of Squid Gam...
3,Late from the trend but still wanna try...my e...
4,There are already more than 10 games on SQUID ...


In [9]:
# To reset the index after remove the duplicate rows
df = df.reset_index(drop=True)
df.head()

,Text
0,Late from the trend but still wanna try...my e...
1,The whole game could have been avoided ☠️ #Squ...
2,I've just watched episode S01 E08 of Squid Gam...
3,Late from the trend but still wanna try...my e...
4,There are already more than 10 games on SQUID ...


In [10]:
import re
def decontracted(phrase):
    # specific
    phrase = re.sub(r"won\'t", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)

    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    return phrase

def listToString_1(s): 
    
    # initialize an empty string
    # print("s = ", s)
    str1 = ""
    count =0    
    # traverse in the string  
    for ele in s:
        if count==0:
            str1 = str1 + str(ele)
            count = count +1
        else:
            str1 = str1 + "" + str(ele)
            count = count + 1
               
    # return string
    #print('str1 = ',str1)
    return str1 
# decontracted

for i in range(len(df['Text'])):
    new_text=''
    text = df['Text'][i]    
    list_dec = decontracted(text)
    new_text = listToString_1(list_dec)
    df['Text'][i] = new_text
df.head()

,Text
0,Late from the trend but still wanna try...my e...
1,The whole game could have been avoided ☠️ #Squ...
2,I have just watched episode S01 E08 of Squid G...
3,Late from the trend but still wanna try...my e...
4,There are already more than 10 games on SQUID ...


In [11]:
# To remove punctuations
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [12]:
def remove_punctuation(text):
    text  = "".join([char for char in text if char not in string.punctuation])
    text = re.sub('[0-9]+', '', text)
    return text

df['absolute_text'] = df['Text'].apply(lambda x: remove_punctuation(x))

In [13]:
# Put the Lower case
df['absolute_text'] = df['absolute_text'].str.lower()

In [14]:
def nospecial(text):
    text = re.sub("[^a-zA-Z0-9]+", " ",text)
    return text
df['absolute_text'] = df['absolute_text'].apply(lambda x: nospecial(x))

In [15]:
df = df[df['absolute_text']!='']
df.head()

,Text,absolute_text
0,Late from the trend but still wanna try...my e...,late from the trend but still wanna trymy edit...
1,The whole game could have been avoided ☠️ #Squ...,the whole game could have been avoided squidga...
2,I have just watched episode S01 E08 of Squid G...,i have just watched episode s e of squid game ...
3,Late from the trend but still wanna try...my e...,late from the trend but still wanna trymy edit...
4,There are already more than 10 games on SQUID ...,there are already more than games on squid gam...


In [16]:
# To remove the stopwords
# downloading stopwords corpus
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
stopwords = set(stopwords.words("english"))

[nltk_data] Downloading package stopwords to C:\Users\acer.LAPTOP-
[nltk_data]     OODPR6CI\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\acer.LAPTOP-
[nltk_data]     OODPR6CI\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to C:\Users\acer.LAPTOP-
[nltk_data]     OODPR6CI\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [17]:
stopwords_set = set(stopwords)
cleaned_tweets_stopwords = []

for index, row in df.iterrows():

    # filerting out all the stopwords 
    words_without_stopwords = [word for word in row.absolute_text.split() if not word in stopwords_set and '#' not in word.lower()]

    # finally creating tweets list of tuples containing stopwords(list) and sentimentType 
    cleaned_tweets_stopwords.append(' '.join(words_without_stopwords))

df['absolute_text'] = cleaned_tweets_stopwords
df.head()

,Text,absolute_text
0,Late from the trend but still wanna try...my e...,late trend still wanna trymy edit entry twice ...
1,The whole game could have been avoided ☠️ #Squ...,whole game could avoided squidgame squidgames
2,I have just watched episode S01 E08 of Squid G...,watched episode e squid game squidgames tvtime...
3,Late from the trend but still wanna try...my e...,late trend still wanna trymy edit entry twice ...
4,There are already more than 10 games on SQUID ...,already games squid games play store squidgame...


In [19]:
# To identify the sentiment of text
from textblob import TextBlob

In [21]:
# To fetch the sentiments using Textblob
def fetch_sentiment_using_textblob(text):
    analysis = TextBlob(text)
    return 'pos' if analysis.sentiment.polarity >= 0 else 'neg'

sentiments_using_textblob = df.absolute_text.apply(lambda tweet: fetch_sentiment_using_textblob(tweet))
pd.DataFrame(sentiments_using_textblob.value_counts())

,absolute_text
pos,123
neg,44


In [25]:
# To add the sentiment in the dataset
df['sentiment'] = sentiments_using_textblob
df.drop('Text',axis=1, inplace=True)
df.head()

,absolute_text,sentiment
0,late trend still wanna trymy edit entry twice ...,neg
1,whole game could avoided squidgame squidgames,neg
2,watched episode e squid game squidgames tvtime...,neg
3,late trend still wanna trymy edit entry twice ...,neg
4,already games squid games play store squidgame...,pos


In [26]:
fetch_sentiment_using_textblob("squid games is bad")

'neg'